# TASK:
1. Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

2. Read the file ( Present approach of reading the file )

3. Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

4. Perform basic validation on data columns : eg: remove special character , white spaces from the col name

5. As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

6. Validate number of columns and column name of ingested file with YAML.

7. Write the file in pipe separated text file (|) in gz format.

8. Create a summary of the file:

- Total number of rows,

- total number of columns

- file size

In [ ]:
###########################
# GENERATING 2GB CSV FILE #
###########################

import csv
import os
import random
import string

SIZE_LIMIT = 2 * 1024 * 1024 * 1024
output_file = "/content/test_2GB.csv"

def random_string(length=10):
    """Generate a random string of fixed length."""
    return ''.join(random.choices(string.ascii_letters, k=length))

def random_float(min_val=0, max_val=1000):
    """Generate a random float rounded to 2 decimal places."""
    return round(random.uniform(min_val, max_val), 2)

# Generate CSV file
with open(output_file, 'w', newline='', buffering=1024*1024) as f:
    writer = csv.writer(f)
    writer.writerow(["ID", "Name", "Value"])

    rows_written = 0
    while True:
        rid = rows_written + 1
        name = random_string(12)
        val = random_float(10, 1000)

        writer.writerow([rid, name, val])
        rows_written += 1

        if rows_written % 100000 == 0:
            f.flush()
            current_size = os.stat(output_file).st_size
            print(f"Rows Written: {rows_written:,}, File Size: {current_size / (1024 * 1024):.2f} MB", flush=True)
            if current_size >= SIZE_LIMIT:
                print("Reached target file size limit.")
                break

print("File generation complete:", output_file)


In [ ]:
import os
# Get the size of the file
file_path = "test_2GB.csv"

file_size = os.path.getsize(file_path)

file_size_mb = file_size / (1024 * 1024)
file_size_gb = file_size / (1024 * 1024 * 1024)

print(f"File Size: {file_size:.2f} bytes")
print(f"File Size: {file_size_mb:.2f} MB")
print(f"File Size: {file_size_gb:.2f} GB")


File Size: 2150048739.00 bytes
File Size: 2050.45 MB
File Size: 2.00 GB


In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Writing testutility.py


# Write the YAML File

In [ ]:
%%writefile config.yaml
file_type: csv
dataset_name: testfile
file_name: test_2GB
table_name: test_table
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - id
    - name
    - value

Writing config.yaml


In [1]:
# Read config file
import testutility as util
config_data = util.read_config_file("config.yaml")
print(config_data)

{'file_type': 'csv', 'dataset_name': 'testfile', 'file_name': 'test_2GB', 'table_name': 'test_table', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['id', 'name', 'value']}


In [ ]:
config_data['inbound_delimiter']

','

In [ ]:
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_2GB',
 'table_name': 'test_table',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['id', 'name', 'value']}

In [2]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re
import dask.dataframe as dd
import time

In [ ]:
!pip install dask[complete] modin[ray] pyyaml

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.2 MB/s eta 0:00:00


# Reading with Pandas

In [ ]:
import pandas as pd
import time

start_time = time.time()
pandas_df = pd.read_csv("/content/test_2GB.csv", delimiter=config_data['inbound_delimiter'], skiprows=config_data['skip_leading_rows'], header=None, names=config_data['columns'])
pandas_read_time = time.time() - start_time
print("Pandas read time:", pandas_read_time, "seconds")


Pandas read time: 217.16818022727966 seconds


# Reading with Dask

In [ ]:
import dask.dataframe as dd
import time

start_time = time.time()

dask_df = dd.read_csv(
    "/content/test_2GB.csv",
    delimiter=config_data['inbound_delimiter'],
    header=0,
    assume_missing=True
)

_ = dask_df.head()

# Step 4: Measure elapsed time
elapsed_time = time.time() - start_time
print(f"Dask read time (including head computation): {elapsed_time:.2f} seconds")


Dask read time (including head computation): 10.50 seconds


# Basic Validation

In [ ]:
# Validate columns with pandas
col_status = util.col_header_val(pandas_df, config_data)
if col_status:
    print("Validation successful with pandas dataframe.")
else:
    print("Validation failed with pandas dataframe.")


column name and column length validation passed
Validation successful with pandas dataframe.


In [9]:
import gzip

input_path = "/content/test_2GB.csv"
output_path = "/content/test_2GB_out.txt.gz"

config_data = util.read_config_file("config.yaml")
expected_col = [c.lower() for c in config_data['columns']]
expected_col.sort()

chunk_size = 10000000

print("Processing and writing in chunks...")
with gzip.open(output_path, 'wt', newline='') as gz_file:
    header_written = False

    for chunk in pd.read_csv(
        input_path,
        delimiter=config_data['inbound_delimiter'],
        header=0,
        chunksize=chunk_size
    ):
        chunk.columns = (
            chunk.columns.str.lower()
            .str.replace(r'[^\w]', '_', regex=True)
            .str.strip('_')
        )

        missing_cols = set(expected_col) - set(chunk.columns)
        if missing_cols:
            raise ValueError(f"Missing columns in chunk: {missing_cols}")

        chunk = chunk[expected_col]

        chunk.to_csv(
            gz_file,
            sep=config_data['outbound_delimiter'],  # Pipe-separated
            index=False,
            header=not header_written
        )
        header_written = True

        print(f"Processed chunk of size {len(chunk)} rows.")

print(f"File written successfully to {output_path}")
print(f"Output file size: {os.path.getsize(output_path) / (1024 * 1024):.2f} MB")


Processing and writing in chunks...
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 10000000 rows.
Processed chunk of size 2500000 rows.
File written successfully to /content/test_2GB_out.txt.gz
Output file size: 1140.53 MB


# Summary of the file

In [12]:
input_path = "/content/test_2GB.csv"

file_size = os.path.getsize(input_path) / (1024 * 1024 * 1024)

print("Reading the file with Dask...")
dask_df = dd.read_csv(
    input_path,
    delimiter=config_data['inbound_delimiter'],
    header=0
)

print("Computing total rows and columns...")
total_rows = dask_df.shape[0].compute()
total_cols = len(dask_df.columns)

# Summary
print("Summary of the original file:")
print("Total number of rows:", total_rows)
print("Total number of columns:", total_cols)
print(f"File size: {file_size:.2f} GB")


Reading the file with Dask...
Computing total rows and columns...
Summary of the original file:
Total number of rows: 72500000
Total number of columns: 3
File size: 2.00 GB
